In [ ]:
!pip install dowhy==0.12 econml==0.15 networkx==3.3

‼️Before running the next cell, press **Ctrl (Cmd) + M + .** and confirm that you want to restsrt the session.

Thank you 🙏🏼

In [ ]:
import dowhy
import econml
print(dowhy.__version__, econml.__version__)

In [ ]:
import warnings
from copy import deepcopy

import numpy as np
import pandas as pd
from scipy import stats

from sklearn.metrics import mean_absolute_percentage_error

import dowhy
from dowhy import CausalModel

from econml.metalearners import SLearner, XLearner, TLearner
from econml.dml import LinearDML, CausalForestDML, DML
from econml.dr import DRLearner, SparseLinearDRLearner

from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

from lightgbm import LGBMRegressor, LGBMClassifier

import networkx as nx

from tqdm import tqdm

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import graphviz

warnings.simplefilter(
    action='ignore',
    category=FutureWarning
)

In [ ]:
COLORS = [
    '#00B0F0',
    '#FF0000',
    '#B0F000'
]

In [ ]:
def plot_effect(effect_true, effect_pred, figsize=(6, 4), ylim=(5000, 22000)):
    plt.figure(figsize=figsize)
    plt.scatter(effect_true, effect_pred, color=COLORS[0])
    plt.plot(np.sort(effect_true), np.sort(effect_true), color=COLORS[1], alpha=.7, label='Perfect model')
    plt.xlabel('$True\ effect$', fontsize=14, alpha=.5)
    plt.ylabel('$Predicted\ effect$', fontsize=14, alpha=.5)
    plt.ylim(ylim[0], ylim[1])
    plt.legend()
    plt.show()

# First Steps in Causal ML



In [ ]:
# Get the data

# Train set
earnings_interaction_train = pd.read_csv(r'https://raw.githubusercontent.com/PacktPublishing/Causal-Inference-and-Discovery-in-Python/refs/heads/main/data/ml_earnings_interaction_train.csv')

# Test set
earnings_interaction_test = pd.read_csv(r'https://raw.githubusercontent.com/PacktPublishing/Causal-Inference-and-Discovery-in-Python/refs/heads/main/data/ml_earnings_interaction_test.csv')

In [ ]:
# Check shapes
earnings_interaction_train.shape, earnings_interaction_test.shape

In [ ]:
# Examine the data

# Train
earnings_interaction_train.head()

In [ ]:
# Test
earnings_interaction_test.head()

## Step 1: Define the Graph

In [ ]:
# Construct the graph
graph = nx.DiGraph()

# Define the edges
edges = [
    ('took_a_course', 'earnings'),
    ('age', 'took_a_course'),
    ('age', 'earnings'),
    ('python_proficiency', 'earnings')
]

# Add edges
graph.add_edges_from(edges)

In [ ]:
# Instantiate the CausalModel object
model = CausalModel(
    data=earnings_interaction_train,
    treatment='took_a_course',
    outcome='earnings',
    effect_modifiers='python_proficiency',
    graph=graph
)

# View the model
model.view_model(size=(4,4))

## Step 2: Identify the Estimand

In [ ]:
# Get the estimand
estimand = model.identify_effect()
print(estimand)

## Step 3: Estimate the Effect

### S-Learner

In [ ]:
# Get estimate (S-Learner)
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.metalearners.SLearner',
    target_units='ate',
    method_params={
        'init_params': {
            'overall_model': LGBMRegressor(n_estimators=500, max_depth=10)
        },
        'fit_params': {}
    })

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.metalearners.SLearner',
    fit_estimator=False,
    target_units=earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1)
).cate_estimates

In [ ]:
# Get the true effect
effect_true = earnings_interaction_test['true_effect'].values

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### T-Learner

In [ ]:
# Get estimate (S-Learner)
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.metalearners.TLearner',
    target_units='ate',
    method_params={
        'init_params': {
            'models': [
                LGBMRegressor(n_estimators=200, max_depth=10),
                LGBMRegressor(n_estimators=200, max_depth=10)
            ]
        },
        'fit_params': {}
    })

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.metalearners.TLearner',
    fit_estimator=False,
    target_units=earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1)
).cate_estimates

In [ ]:
# Get the true effect
effect_true = earnings_interaction_test['true_effect'].values

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### X-Learner

In [ ]:
# Get estimate (X-Learner)
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.metalearners.XLearner',
    target_units='ate',
    method_params={
        'init_params': {
            'models': LGBMRegressor(n_estimators=50, max_depth=10)
        },
        'fit_params': {}
    })

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.metalearners.XLearner',
    fit_estimator=False,
    target_units=earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1)
).cate_estimates

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### Doubly Robust Learner (DR-Learner)

In [ ]:
# Get estimate (Doubly robust)
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dr.LinearDRLearner',
    target_units='ate',
    method_params={
        'init_params': {
            'model_propensity': LogisticRegression(),
            'model_regression': LGBMRegressor(n_estimators=1000, max_depth=10)
        },
        'fit_params': {}
    })

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dr.LinearDRLearner',
    fit_estimator=False,
    target_units=earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1)
).cate_estimates

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### Non-linear DR

In [ ]:
# Get estimate (Doubly robust non-linear)
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dr.DRLearner',
    target_units='ate',
    method_params={
        'init_params': {
            'model_propensity': LogisticRegression(),
            'model_regression': LGBMRegressor(n_estimators=1000, max_depth=10),
            'model_final': LGBMRegressor(n_estimators=500, max_depth=10),
        },
        'fit_params': {}
    })

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dr.LinearDRLearner',
    fit_estimator=False,
    target_units=earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1)
).cate_estimates

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### DML (Double Machine Learning)

In [ ]:
# Get estimate (DML)
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dml.LinearDML',
    target_units='ate',
    method_params={
        'init_params': {
            'model_y': LGBMRegressor(n_estimators=500, max_depth=10),
            'model_t': LogisticRegression(),
            'discrete_treatment': True
        },
        'fit_params': {}
    })

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dml.LinearDML',
    fit_estimator=False,
    target_units=earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1)
).cate_estimates

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

### DML with Cross-Validation

In [ ]:
# Define wrapped CV models
model_y = GridSearchCV(
    estimator=LGBMRegressor(),
    param_grid={
        'max_depth': [3, 10, 20, 100],
        'n_estimators': [10, 50, 100]
    }, cv=10, n_jobs=-1, scoring='neg_mean_squared_error'
)

model_t = GridSearchCV(
    estimator=LGBMClassifier(),
    param_grid={
        'max_depth': [3, 10, 20, 100],
        'n_estimators': [10, 50, 100]
    }, cv=10, n_jobs=-1, scoring='accuracy'
)

In [ ]:
# Estimate the effect
estimate = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dml.LinearDML',
    target_units='ate',
    method_params={
        'init_params': {
            'model_y': model_y,
            'model_t': model_t,
            'discrete_treatment': True,
            'cv': 4
        },
        'fit_params': {
        }
    })

In [ ]:
estimate.cate_estimates.mean()

In [ ]:
# Compute predictions
effect_pred = model.estimate_effect(
    identified_estimand=estimand,
    method_name='backdoor.econml.dml.LinearDML',
    fit_estimator=False,
    target_units=earnings_interaction_test.drop(['true_effect', 'took_a_course'], axis=1)
).cate_estimates

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
)

## Using Meta-Learners Directly

### Read and clean the data

In [ ]:
# Get the data
hillstrom_clean = pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Causal-Inference-and-Discovery-in-Python/refs/heads/main/data/hillstrom_clean.csv')

# Define label mappings
labels_mapping = {"control": 0, "womans_email": 1, "mens_email": 2}


# Drop redundant cols to avoid multicollinearity
hillstrom_clean = hillstrom_clean.drop(['zip_code__urban', 'channel__web'], axis=1)

# Split data
hillstrom_X = hillstrom_clean.drop(['visit', 'conversion', 'spend', 'treatment'], axis=1)
hillstrom_Y = hillstrom_clean['spend']
hillstrom_T = hillstrom_clean['treatment']


In [ ]:
# Train test split
X_train, X_test, y_train, y_test, T_train, T_test = train_test_split(
    hillstrom_X,
    hillstrom_Y,
    hillstrom_T,
    test_size=.5
)

In [ ]:
# How many observations in train/test converted?
(y_train[T_train > 0] > 0).sum(), (y_test[T_test > 0] > 0).sum()

In [ ]:
# Define the base model
base_model = LGBMRegressor(
    n_estimators=100,
    max_depth=10,
    learning_rate=0.1,
    )

# Instantiate the S-Learner
s_learner = SLearner(
    overall_model=base_model
)

In [ ]:
# Fit the model
s_learner.fit(
    Y=y_train,
    T=T_train,
    X=X_train
)

In [ ]:
# Predict the effects
uplifts = s_learner.effect(
    T0=0,
    T1=1,
    X=X_test.values
)

uplifts

## Your Turn: Who Should We Target?

Use DML or DRLearner with base learners of your choice, train them on Hillstrom data, generate predictions on the test set and answer the question: does sending mens or womens mail have greater impact on **newbie visitors' spending**?